# Azure AI Content Understanding (public preview)

This service is available in the Azure AI Foundry portal and through REST API.

Supported content type:
- Documents and forms
- Images
- Audio
- Video

> Personal note:
>
> It seems like Azure made this new service by combining their existing services:
> - AI Document Intelligence: for document, form, and image analysis.
> - AI Video Indexer: for audio and video analysis.

The service provides some pre-built analyzers:
- For content ingestion:
    - `prebuilt-documentAnalyzer`: extracts text and layout from document and images, produces summary.
    - `prebuilt-imageAnalyzer`: generates image caption.
    - `prebuilt-audioAnalyzer`: transcribes audio, performs speaker diarization, produces summary.
    - `prebuilt-videoAnalyzer`: transcribes video, identifies keyframes/camera shots, segments video, generate summary for the segments.
- For intelligent document processing:
    - `prebuilt-invoice`: extracts text/layout, structured data from invoice documents/images.
    - `prebuilt-callCenter`: transcribes, diarizes speakers, generates summaries, customer sentiment analysis, discussion topics.


# Setup

In [66]:
import os
import requests
import io
import json

from dotenv import load_dotenv
from urllib.parse import urljoin

load_dotenv()

CU_SERVICE_ENDPOINT = os.getenv("CU_SERVICE_ENDPOINT")
CU_SERVICE_KEY = os.getenv("CU_SERVICE_KEY")

# Usage

## Document analysis

Experiment on an electric bill image

<img src="https://thanhnien.mediacdn.vn/Uploaded/hongky-qc/2022_07_02/mau-hoa-don-tien-dien-moi-9838.jpg" alt="The electric bill" width="400"/>


In [70]:
image_url = "https://thanhnien.mediacdn.vn/Uploaded/hongky-qc/2022_07_02/mau-hoa-don-tien-dien-moi-9838.jpg"
analyzer = "prebuilt-invoice"
api_version = "2025-05-01-preview"

doc_cu_response = requests.post(
    url=str(urljoin(CU_SERVICE_ENDPOINT, f"/contentunderstanding/analyzers/{analyzer}:analyze?api-version={api_version}")),
    headers={
        "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY,
        # "Content-Type": "application/json"
    },
    data=json.dumps({
        "url": image_url
    })
)
print(doc_cu_response.headers)
print(doc_cu_response.json())

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'request-id': 'bbdf50bc-7b13-4857-a1be-e5629d7ba663', 'x-ms-request-id': 'bbdf50bc-7b13-4857-a1be-e5629d7ba663', 'Operation-Location': 'https://ai-buiminhquanhcmus-3831-westus.cognitiveservices.azure.com/contentunderstanding/analyzerResults/bbdf50bc-7b13-4857-a1be-e5629d7ba663?api-version=2025-05-01-preview', 'api-supported-versions': '2024-12-01-preview,2025-05-01-preview', 'x-envoy-upstream-service-time': '8064', 'apim-request-id': 'bbdf50bc-7b13-4857-a1be-e5629d7ba663', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:25:14 GMT'}
{'id': 'bbdf50bc-7b13-4857-a1be-e5629d7ba663', 'status': 'Running', 'result': {'analyzerId': 'prebuilt-invoice', 'apiVersion': '2025-05-01-preview', 'createdAt': '2025-06-28T04:25:07Z', 'warnings': [], 'contents': []}}


In [71]:
id = doc_cu_response.json()["id"]
urljoin(CU_SERVICE_ENDPOINT, f"/contentunderstanding/analyzers/{analyzer}/results/{id}?api-version={api_version}")

'https://ai-buiminhquanhcmus-3831-westus.cognitiveservices.azure.com/contentunderstanding/analyzers/prebuilt-invoice/results/bbdf50bc-7b13-4857-a1be-e5629d7ba663?api-version=2025-05-01-preview'

In [72]:
# Implemented following the 'Use the Content Understanding REST API' 
# at https://learn.microsoft.com/en-us/training/modules/analyze-content-ai/04-use-api

# Not working, wrong URL
doc_cu_status = requests.get(
    url=urljoin(CU_SERVICE_ENDPOINT, f"/contentunderstanding/analyzers/{analyzer}/results/{id}?api-version={api_version}"),
    headers={
        "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY
    },
)
print(doc_cu_status.json())

{'error': {'code': '404', 'message': 'Resource not found'}}


In [73]:
# Implemented following the 'Quickstart: Use Azure AI Content Understanding REST API' 
# at https://learn.microsoft.com/en-us/azure/ai-services/content-understanding/quickstart/use-rest-api?tabs=document

# Not working, wrong URL
doc_cu_status = requests.get(
    url=urljoin(CU_SERVICE_ENDPOINT, f"/contentunderstanding/analyzers/{id}?api-version={api_version}"),
    headers={
        "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY
    },
)
print(doc_cu_status.json())

{'error': {'code': 'NotFound', 'message': 'Resource not found.', 'innererror': {'code': 'ModelNotFound', 'message': 'The requested model was not found.'}}}


In [76]:
# Let's try using the 'Operation-Location' in the headers

doc_cu_status = requests.get(
    url=doc_cu_response.headers["Operation-Location"],
    headers={
        "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY
    },
)

print(doc_cu_status.headers)
print(doc_cu_status.json())

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'request-id': '4c5799ee-fe36-44f9-9479-fb7c49acbe17', 'x-ms-request-id': '4c5799ee-fe36-44f9-9479-fb7c49acbe17', 'api-supported-versions': '2024-12-01-preview,2025-05-01-preview', 'x-envoy-upstream-service-time': '27', 'apim-request-id': '4c5799ee-fe36-44f9-9479-fb7c49acbe17', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:25:44 GMT'}
{'id': 'bbdf50bc-7b13-4857-a1be-e5629d7ba663', 'status': 'Running', 'result': {'analyzerId': 'prebuilt-invoice', 'apiVersion': '2025-05-01-preview', 'createdAt': '2025-06-28T04:25:07Z', 'warnings': [], 'contents': []}}


The document is being processed, let's wait and try again in a few seconds.

In [80]:
doc_cu_status = requests.get(
    url=doc_cu_response.headers["Operation-Location"],
    headers={
        "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY
    },
)

print(doc_cu_status.headers)
print(doc_cu_status.json())

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'request-id': 'bc87551e-2495-4fc7-995e-be3b84b6bf80', 'x-ms-request-id': 'bc87551e-2495-4fc7-995e-be3b84b6bf80', 'api-supported-versions': '2024-12-01-preview,2025-05-01-preview', 'x-envoy-upstream-service-time': '29', 'apim-request-id': 'bc87551e-2495-4fc7-995e-be3b84b6bf80', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:27:40 GMT'}
{'id': 'bbdf50bc-7b13-4857-a1be-e5629d7ba663', 'status': 'Running', 'result': {'analyzerId': 'prebuilt-invoice', 'apiVersion': '2025-05-01-preview', 'createdAt': '2025-06-28T04:25:07Z', 'warnings': [], 'contents': []}}


The process failed multiple times. Let's try using the FormRecognizer SDK

In [54]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

da_client = DocumentAnalysisClient(
    endpoint=CU_SERVICE_ENDPOINT, 
    credential=AzureKeyCredential(CU_SERVICE_KEY)
)

In [55]:
response = da_client.begin_analyze_document_from_url(analyzer, image_url)
result = response.result()

In [57]:
for idx, document in enumerate(result.documents):
    print("--------Analyzing document #{}--------".format(idx + 1))
    print("Document has type {}".format(document.doc_type))
    print("Document has confidence {}".format(document.confidence))
    print("Document was analyzed by model with ID {}".format(result.model_id))
    for name, field in document.fields.items():
        field_value = field.value if field.value else field.content
        print("Found field '{}' with value '{}' and with confidence {}".format(name, field_value, field.confidence))

--------Analyzing document #1--------
Document has type invoice
Document has confidence 1.0
Document was analyzed by model with ID prebuilt-invoice
Found field 'CustomerAddress' with value 'AddressValue(house_number=None, po_box=None, road=None, city=None, state=None, postal_code=None, country_region=None, street_address=, unit=None, city_district=None, state_district=None, suburb=Nguyễn Trãi, house=None, level=None)' and with confidence 0.926
Found field 'CustomerAddressRecipient' with value 'BÌNH' and with confidence 0.839
Found field 'CustomerName' with value 'BÌNH' and with confidence 0.839
Found field 'InvoiceDate' with value 'Ngày (Date) 29 tháng (month) 06 năm (year) 2022' and with confidence 0.689
Found field 'InvoiceId' with value '7166' and with confidence 0.957
Found field 'InvoiceTotal' with value '1018449.0' and with confidence 0.95
Found field 'Items' with value '[DocumentField(value_type=dictionary, value={'Amount': DocumentField(value_type=currency, value=CurrencyValue(

This service has been updated over time:

Form Recognizer (old version) -> AI Document Intelligence (GA) -> Content Understanding (preview)

## Audio analysis

<video controls src="https://github.com/MicrosoftLearning/mslearn-ai-information-extraction/raw/refs/heads/main/Instructions/Labs/media/call-1.mp4" title="Call 1" width="300">
    <track src="https://github.com/MicrosoftLearning/mslearn-ai-information-extraction/raw/refs/heads/main/Instructions/Labs/media/call-1.vtt" kind="captions" srclang="en" label="English">
</video>


In [67]:
audio_url = "https://github.com/MicrosoftLearning/mslearn-ai-information-extraction/raw/refs/heads/main/Instructions/Labs/media/call-1.mp4"
audio_analyzer = "prebuilt-audioAnalyzer"
api_version = "2025-05-01-preview"

audio_cu_response = requests.post(
    url=str(urljoin(CU_SERVICE_ENDPOINT, f"/contentunderstanding/analyzers/{audio_analyzer}:analyze?api-version={api_version}")),
    headers={
        "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY,
        # "Content-Type": "application/json"
        "x-ms-useragent": "azure-ai102-labs"
    },
    data=json.dumps({
        "url": audio_url
    })
)
print(audio_cu_response.headers)
print(audio_cu_response.json())

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'request-id': '4c2c6c46-8390-4a0a-984a-518d27d4a9f9', 'x-ms-request-id': '4c2c6c46-8390-4a0a-984a-518d27d4a9f9', 'Operation-Location': 'https://ai-buiminhquanhcmus-3831-westus.cognitiveservices.azure.com/contentunderstanding/analyzerResults/4c2c6c46-8390-4a0a-984a-518d27d4a9f9?api-version=2025-05-01-preview', 'api-supported-versions': '2024-12-01-preview,2025-05-01-preview', 'x-envoy-upstream-service-time': '1496', 'apim-request-id': '4c2c6c46-8390-4a0a-984a-518d27d4a9f9', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:24:29 GMT'}
{'id': '4c2c6c46-8390-4a0a-984a-518d27d4a9f9', 'status': 'Running', 'result': {'analyzerId': 'prebuilt-audioAnalyzer', 'apiVersion': '2025-05-01-preview', 'createdAt': '2025-06-28T04:24:29Z', 'warnings': [], 'contents': []}}


In [103]:
# Let's try getting the result using the 'Operation-Location' in the headers

audio_result = requests.get(
    url=audio_cu_response.headers["Operation-Location"],
    headers={
        "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY
    },
)

print(audio_result.headers)
print(audio_result.json())

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'request-id': 'c19f7377-b25d-4999-8a26-a0c542716217', 'x-ms-request-id': 'c19f7377-b25d-4999-8a26-a0c542716217', 'api-supported-versions': '2024-12-01-preview,2025-05-01-preview', 'x-envoy-upstream-service-time': '70', 'apim-request-id': 'c19f7377-b25d-4999-8a26-a0c542716217', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:57:16 GMT'}
{'id': '4c2c6c46-8390-4a0a-984a-518d27d4a9f9', 'status': 'Failed', 'error': {'code': 'InvalidRequest', 'message': 'Invalid Request.', 'innererror': {'code': 'InternalServerError', 'message': 'An unknown error was encountered. Please contact us at https://aka.ms/cu-feedback?requestId=4c2c6c46-8390-4a0a-984a-518d27d4a9f9.'}}, 'result': {'analyzerId': 'prebuilt-audioAnalyzer', 'apiVersion': '2025-05-01-preview', 'createdAt': '2025-06-28T04:24:29Z', 'warnings': [], 'conte

Currently there is no official SDK for VideoIndexer from Azure, so the audio analysis experiment will be left as a failed result.

In [104]:
# TODO(quanbui): Implement audio/video analysis using the old VideoIndexer API. 
# Reference: https://github.dev/Azure-Samples/azure-video-indexer-samples

## Create a custom analyzer

I'll create a business card analyzer to extract name and email address

In [78]:
custom_analyzer_schema = {
    "description": "Simple business card",
    "baseAnalyzerId": "prebuilt-documentAnalyzer",
    "config": {
        "returnDetails": True
    },
    "fieldSchema": {
        "fields": {
            "ContactName": {
                "type": "string",
                "method": "extract",
                "description": "Name on business card"
            },
            "EmailAddress": {
                "type": "string",
                "method": "extract",
                "description": "Email address on business card"
            }
        }
    }
}

custom_analyzer_name = "business_card_analyzer"

In [79]:
creation_response = requests.put(url=urljoin(CU_SERVICE_ENDPOINT, f"contentunderstanding/analyzers/{custom_analyzer_name}?api-version=2025-05-01-preview"),
                                 headers={
                                     "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY
                                 },
                                 data=json.dumps(custom_analyzer_schema))

print(creation_response.headers)
print(creation_response.json())

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'request-id': 'e02efecd-bd70-4039-8c9a-d13d143c2945', 'x-ms-request-id': 'e02efecd-bd70-4039-8c9a-d13d143c2945', 'Operation-Location': 'https://ai-buiminhquanhcmus-3831-westus.cognitiveservices.azure.com/contentunderstanding/analyzers/business_card_analyzer/operations/e02efecd-bd70-4039-8c9a-d13d143c2945?api-version=2025-05-01-preview', 'api-supported-versions': '2025-05-01-preview', 'x-envoy-upstream-service-time': '59', 'apim-request-id': 'e02efecd-bd70-4039-8c9a-d13d143c2945', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:26:38 GMT'}
{'analyzerId': 'business_card_analyzer', 'description': 'Simple business card', 'createdAt': '2025-06-28T04:26:38Z', 'lastModifiedAt': '2025-06-28T04:26:38Z', 'baseAnalyzerId': 'prebuilt-documentAnalyzer', 'config': {'returnDetails': True, 'enableOcr': True, 'enabl

Let's try it out with a sample card

<img src="https://github.com/MicrosoftLearning/mslearn-ai-information-extraction/raw/refs/heads/main/Labfiles/content-app/biz-card-1.png" alt="Sample business card" width="400"/>

In [82]:
biz_card_url = "https://github.com/MicrosoftLearning/mslearn-ai-information-extraction/raw/refs/heads/main/Labfiles/content-app/biz-card-1.png"

biz_card = requests.get(biz_card_url).content

bc_response = requests.post(
    url=urljoin(CU_SERVICE_ENDPOINT, f"contentunderstanding/analyzers/{custom_analyzer_name}:analyze?api-version={api_version}"),
    headers={
        "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY,
        "Content-Type": "application/octet-stream"
    },
    data=io.BytesIO(biz_card)
)

print(bc_response.headers)
print(bc_response.json())

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'request-id': 'ec680768-657b-4725-ba60-20b420b55cad', 'x-ms-request-id': 'ec680768-657b-4725-ba60-20b420b55cad', 'Operation-Location': 'https://ai-buiminhquanhcmus-3831-westus.cognitiveservices.azure.com/contentunderstanding/analyzerResults/ec680768-657b-4725-ba60-20b420b55cad?api-version=2025-05-01-preview', 'api-supported-versions': '2024-12-01-preview,2025-05-01-preview', 'x-envoy-upstream-service-time': '326', 'apim-request-id': 'ec680768-657b-4725-ba60-20b420b55cad', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:33:24 GMT'}
{'id': 'ec680768-657b-4725-ba60-20b420b55cad', 'status': 'Running', 'result': {'analyzerId': 'business_card_analyzer', 'apiVersion': '2025-05-01-preview', 'createdAt': '2025-06-28T04:33:25Z', 'warnings': [], 'contents': []}}


In [83]:
# Fetching the result

bc_result = requests.get(
    url=bc_response.headers["Operation-Location"],
    headers={"Ocp-Apim-Subscription-Key": CU_SERVICE_KEY})

print(bc_result.headers)
print(bc_result.json())

{'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json', 'request-id': 'a32e4e54-7c24-4a14-96f9-76c038b5c48a', 'x-ms-request-id': 'a32e4e54-7c24-4a14-96f9-76c038b5c48a', 'api-supported-versions': '2024-12-01-preview,2025-05-01-preview', 'x-envoy-upstream-service-time': '32', 'apim-request-id': 'a32e4e54-7c24-4a14-96f9-76c038b5c48a', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:34:31 GMT'}
{'id': 'ec680768-657b-4725-ba60-20b420b55cad', 'status': 'Succeeded', 'result': {'analyzerId': 'business_card_analyzer', 'apiVersion': '2025-05-01-preview', 'createdAt': '2025-06-28T04:33:25Z', 'warnings': [], 'contents': [{'markdown': '<figure>\n</figure>\n\n\nAdventure Works Cycles\n\nRoberto Tamburello\nEngineering Manager\n\nroberto@adventure-works.com\n\n555-123-4567\n', 'fields': {'ContactName': {'type': 'string', 'valueString': 'Roberto Tamburello'}, 'EmailAddr

In [84]:
bc = bc_result.json()["result"]
bc

{'analyzerId': 'business_card_analyzer',
 'apiVersion': '2025-05-01-preview',
 'createdAt': '2025-06-28T04:33:25Z',
 'warnings': [],
 'contents': [{'markdown': '<figure>\n</figure>\n\n\nAdventure Works Cycles\n\nRoberto Tamburello\nEngineering Manager\n\nroberto@adventure-works.com\n\n555-123-4567\n',
   'fields': {'ContactName': {'type': 'string',
     'valueString': 'Roberto Tamburello'},
    'EmailAddress': {'type': 'string',
     'valueString': 'roberto@adventure-works.com'}},
   'kind': 'document',
   'startPageNumber': 1,
   'endPageNumber': 1,
   'unit': 'pixel',
   'pages': [{'pageNumber': 1,
     'angle': 0.0234212,
     'width': 1000,
     'height': 620,
     'spans': [{'offset': 0, 'length': 127}],
     'words': [{'content': 'Adventure',
       'span': {'offset': 21, 'length': 9},
       'confidence': 0.994,
       'source': 'D(1,283,87,543,87,542,140,282,138)'},
      {'content': 'Works',
       'span': {'offset': 31, 'length': 5},
       'confidence': 0.997,
       'source

In [94]:
print("Content:\n", bc["contents"][0]["markdown"])
print()

print("Contact name:", bc["contents"][0]["fields"]["ContactName"]["valueString"])
print("Contact email:", bc["contents"][0]["fields"]["EmailAddress"]["valueString"])

Content:
 <figure>
</figure>


Adventure Works Cycles

Roberto Tamburello
Engineering Manager

roberto@adventure-works.com

555-123-4567


Contact name: Roberto Tamburello
Contact email: roberto@adventure-works.com


Clean up resources

In [ ]:
deletion_response = requests.put(url=urljoin(CU_SERVICE_ENDPOINT, f"contentunderstanding/analyzers/{custom_analyzer_name}?api-version=2025-05-01-preview"),
                                 headers={
                                     "Ocp-Apim-Subscription-Key": CU_SERVICE_KEY
                                 },
                                 # The Azure API Reference missed the request body specification 
                                 # (https://learn.microsoft.com/en-us/rest/api/contentunderstanding/content-analyzers/delete?view=rest-contentunderstanding-2025-05-01-preview&tabs=HTTP)
                                 # So for now it could not be deleted.
                                 data=json.dumps({
                                     "baseAnalyzerId": "prebuilt-documentAnalyzer"
                                 }))

print(deletion_response.headers)
print(deletion_response.status_code)
print(deletion_response.json())

{'Content-Length': '236', 'Content-Type': 'application/json; charset=utf-8', 'request-id': 'df27af33-922d-477b-8063-5f9a9b72c20a', 'x-ms-request-id': 'df27af33-922d-477b-8063-5f9a9b72c20a', 'x-ms-error-code': 'Conflict', 'api-supported-versions': '2025-05-01-preview', 'x-envoy-upstream-service-time': '20', 'apim-request-id': 'df27af33-922d-477b-8063-5f9a9b72c20a', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'x-content-type-options': 'nosniff', 'x-ms-region': 'West US', 'Date': 'Sat, 28 Jun 2025 04:54:06 GMT'}
409
{'error': {'code': 'Conflict', 'message': 'The request could not be completed due to a conflict with the current state of the target resource.', 'innererror': {'code': 'ModelExists', 'message': 'A model with the provided name already exists.'}}}
